In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam

import numpy as np

In [182]:

class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation = None, **kwargs):
        self.dense = dense 
        self.activation = tf.keras.activations.get(activation)
        super(DenseTranspose, self).__init__(**kwargs)

    def build(self, batch_input_shape):
        self.b = self.add_weight(name= "bias", shape = [ self.dense.input_shape[-1]], initializer = "zeros")
        self.w = self.dense.weights[0]
        super().build(batch_input_shape)
        
        
    def call(self, inputs):
        z = tf.linalg.matmul(inputs, self.w, transpose_b = True)
        return self.activation(z + self.b)

    
    def get_weights(self):
        return {"w": np.shape(tf.transpose(self.w))}    

    @property 
    def weights_transpose(self):
        return tf.transpose(self.dense.weights[0])


class Encoder(tf.keras.layers.Layer):

    def __init__(self, input_shape, dropout, reg_cof, compression, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self._output_size = int(input_shape * compression)
        self.input_encoder =  Input(shape = (input_shape,))
        self.drop_encoder = Dropout(dropout)
        self.dense_encoder = Dense(self._output_size,activation="tanh",kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
        bias_initializer=tf.keras.initializers.Zeros(),kernel_regularizer=tf.keras.regularizers.l1(reg_cof))
    
    def build(self, input_shape):
        super().build(input_shape)

    def call(self, inputs):
        x = self.drop_encoder(inputs) 
        x = self.dense_encoder(x)
        return x

    def get_dense_encoder(self):
        return self.dense_encoder

    @property
    def output_size(self):
        return self._output_size 

class Decoder(tf.keras.layers.Layer):

    def __init__(self, encoder, **kwargs ):
        self.encoder_d = encoder.get_dense_encoder() 
        self.dense_decoder = DenseTranspose(dense = self.encoder_d)
        super(Decoder, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.dense_decoder.build(input_shape)
        super().build(input_shape)
        print(self.dense_decoder.get_weights())

        

    def call(self, inputs):
        x = self.dense_decoder(inputs)
        return x


In [174]:

class Autoencoder(tf.keras.Model):

    def __init__(self, input_size=32, compression = 0.8, dropout = 0.2, reg_cof = 0.001, **kwargs):
        super(Autoencoder, self).__init__(**kwargs)
        self.input_size = input_size
        self.compression = 0.8
        self.dropout = dropout
        self.reg_cof = reg_cof 
        encoder = Encoder(
            input_shape = self.input_size,
            dropout = self.dropout, 
            reg_cof = self.reg_cof,
            compression = self.compression, 
            )
        encoder.build(input_size)
        encoder((np.random.rand(32)/1000,))
        print(encoder.weights)
        self.encoder = encoder
        self.decoder = Decoder(
            encoder = self.encoder
            ) 
        

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.decoder(x)
        return x

    def encode(self, X):
        return self.encoder(X)

    def decode(self, Z):
        return self.decoder(Z)


In [195]:
encoder = Encoder(input_shape=32, dropout=0.2, reg_cof=0.001, compression=0.8)
encoder((trainY[0],))
decoder = DenseTranspose(encoder)

opt = Adam(lr=0.01)
model.compile(loss='mse', optimizer=opt)
model = tf.keras.models.Model(encoder, decoder(encoder))
model.compile()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



AttributeError: The layer has never been called and thus has no defined input shape.

In [183]:
model = Autoencoder()
opt = Adam(lr=0.01)
model.compile(loss='mse', optimizer=opt)
model.fit(
    trainY, trainY, 
    epochs =2)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[<tf.Variable 'encoder_99/dense_99/kernel:0' shape=(32, 25) dtype=float32, numpy=
array([[-5.62692573e-03, -3.23699345e-03, -9.17229056e-03,
        -1.12210959e-02, -2.25413218e-03,  3.96442739e-03,
         4.50964691e-03, -2.18483377e-02, -3.46395164e-03,
        -2.01933886e-04,  1.26074059e-02, -7.82557018e-03,
        -5.06875524e-03,  3.59702785e-03,  1.06224036e-02,
         3.66191985e-03,  2.62866961e-03, -3.60417576e-03,
        -5.56925125e-03, -1.11007188e-02,  2.86216033e-03,
        -6.53

AttributeError: The layer has never been called and thus has no defined input shape.

In [302]:
import util as ut
trainY = ut.generate_training_sat(32,100)[1]
np.shape(trainY[0])


(32,)

In [354]:
sample_space = 10
epochs = 10


model = shallowNet.build()
opt = Adam(lr=0.01)
model.compile(loss='mse', optimizer=opt)
model.fit(
    trainY, trainY, 
    epochs = 2, 
    batch_size = 1,
    callbacks=[LossAndErrorPrintingCallback()]
)


Model: "model_95"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_157 (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
dropout_133 (Dropout)        (None, 32)                0         
_________________________________________________________________
dense_133 (Dense)            (None, 25)                825       
_________________________________________________________________
dense_transpose_73 (DenseTra (None, 32)                857       
Total params: 857
Trainable params: 857
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
For batch 0, loss is    0.50.
  1/100 [..............................] - ETA: 0s - loss: 0.5033For batch 1, loss is    0.55.
For batch 2, loss is    0.61.
For batch 3, loss is    0.60.
For batch 4, loss is    0.59.
For batch 5, loss is    0.57.
For

IndexError: tuple index out of range

In [290]:
features_list_encoder = [layer.output for layer in model.layers[:3]]
encoder = tf.keras.Model(inputs = model.input, outputs = features_list_encoder)
features_list_decoder = model.layers[3]# [layer.output for layer in model.layers[3]]
inputs = Input(encoder.layers[-1].output_shape[-1])
features_list_decoder(inputs)
decoder = tf.keras.Model( inputs = inputs ,outputs = features_list_decoder(inputs))
encoder.summary()
decoder.summary()
decoder.output


Model: "model_74"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_114 (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
dropout_113 (Dropout)        (None, 32)                0         
_________________________________________________________________
dense_113 (Dense)            (None, 25)                825       
Total params: 825
Trainable params: 825
Non-trainable params: 0
_________________________________________________________________
Model: "model_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_137 (InputLayer)       [(None, 25)]              0         
_________________________________________________________________
dense_transpose_53 (DenseTra (None, 32)                857       
Total params: 857
Trainable params: 857
No

<tf.Tensor 'dense_transpose_53_8/Identity:0' shape=(None, 32) dtype=float32>

In [1]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()

epochs =2
for epoch in range(epochs):
    print('Start of epoch %d' % (epochs,))
    # iterate over the batches of the dataset 
    for  x_batch_train in trainY:
        with tf.GradientTape() as tape:
            reconstructed = encoder((x_batch_train,))
            print(reconstructed)
            loss = mse_loss_fn(x_batch_train, reconstructed)
            los += sum(model.losses)

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        loss_metric(loss)

        if step % 1 == 0:
            print('step %s: mean loss = %s' % (step, loss_metric.result()))

NameError: name 'tf' is not defined